Laden der erforderlichen Bibliotheken

In [ ]:
import tensorflow as tf
keras = tf.keras
from keras import losses, layers, models, metrics, optimizers, callbacks
from matplotlib import pyplot as plt
import skimage
import numpy as np

from picamera2 import Picamera2
camera = Picamera2()
classes = []
data = []
labels = []

Enter name for class

In [ ]:
label = input("Class name:")
if label not in classes:
    classes.append(label)

Capture images for class - use multiple times to capture many images

In [ ]:
camera.start()
image = camera.capture_array()
camera.stop()
# take green channel
img = image[:,:,0:3]
plt.imshow(img)
plt.show()
data.append(img)
labels.append(classes.index(label))

Plot the dataset

In [ ]:
# plot dataset as image matrix
fig, axes = plt.subplots(5, len(data) // 5 + 1, figsize=(len(data) // 5 + 3, 7))
# flatten axes
axes = axes.flatten()
for i, ax in enumerate(axes):
    if i < len(data):
        ax.imshow(data[i])
        ax.set_title(classes[labels[i]])
    ax.axis('off')
plt.show()

In [ ]:
_data = np.stack(data)
_labels = np.array(labels)

#shuffle data
splits = np.random.permutation(len(_data))
_data = _data[splits]
_labels = _labels[splits]
_labels = tf.one_hot(_labels, len(classes))

#split into test and train data
data_train = _data[:int(len(data) * 0.8)]
labels_train = _labels[:int(len(data) * 0.8)]
data_test = _data[int(len(data) * 0.8):]
labels_test = _labels[int(len(data) * 0.8):]

# create a network that can separate the two classes

network = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(None, None, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Lambda(tf.reduce_mean, arguments={'axis': [1, 2]}),
    layers.Flatten(),
    layers.Dropout(0.2),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(2, activation='softmax')
])
network.summary()
network.compile(loss = losses.MeanSquaredError(), metrics=[metrics.CategoricalAccuracy()], optimizer=optimizers.Adam())

training = network.fit(data_train, labels_train, epochs=100, batch_size=128, validation_data=(data_test, labels_test), 
                       callbacks=[callbacks.EarlyStopping(patience=5, monitor='val_loss', restore_best_weights=True)])

Plot learning curve

In [ ]:
# show loss and accuracy in twin x plots
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
ax1.plot(training.history['loss'], label='loss', color='red')
ax1.plot(training.history['val_loss'], label='val_loss', color='orange')
ax1.set_xlabel('epoch')
ax1.set_ylabel('loss')
ax1.legend()
ax2.plot(training.history['categorical_accuracy'], label='accuracy', color='blue')	
ax2.plot(training.history['val_categorical_accuracy'], label='val_accuracy', color='green')
ax2.set_ylabel('accuracy')
ax2.legend()
plt.show()

Test model

In [ ]:
camera.start()
image = camera.capture_array()
camera.stop()
# take green channel
img = image[:,:,0:3]
plt.imshow(img)
plt.show()

#invoke network
pred = network.predict(np.array([img]))
print(classes[np.argmax(pred)])

#calculate confidence
conf = 1 - tf.reduce_sum(-pred * tf.math.log(pred + 0.0001), axis=-1)
print(conf)